In [9]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

In [3]:
df = pd.read_csv("Data/Interpolated_soil_combined/combined_zones.csv")

# Define the function that will assign the soil test
def assign_soil_test(field_id):
    if field_id in ['16A', '6-12', 'SW16']:
        return 'Mehlich'
    elif field_id in ['Y10', 'Y8']:
        return 'Haney'
    else:
        return None  # or some default value

# Create the new column
df['Soil_Test'] = df['FieldID'].apply(assign_soil_test)

rainfall = {2018: 350, 
            2019: 505,
            2020: 481,
            2022: 279}

df['Rainfall'] = df['Year'].map(rainfall)

df = df[['Longitude', 'Latitude',
       'Yield', 'Moisture', 'PlotID', 'FieldID', 'TOC_0_2', 'IC_0_2', 'TN_0_2', 
                           'DI_Al_0_2', 'DI_Ca_0_2', 'DI_Fe_0_2', 'DI_K_0_2', 'DI_Mg_0_2', 'DI_P_0_2', 
                           'H3A_Al_0_2', 'H3A_Ca_0_2', 'H3A_Fe_0_2', 'H3A_K_0_2', 'H3A_Mg_0_2', 'H3A_P_0_2',
                           'M3_Al_0_2', 'M3_Ca_0_2', 'M3_Fe_0_2', 'M3_K_0_2', 'M3_Mg_0_2', 'M3_P_0_2',
                           'Ols_Al_0_2', 'Ols_Ca_0_2', 'Ols_Fe_0_2', 'Ols_K_0_2', 'Ols_Mg_0_2', 'Ols_P_0_2',
                           'TOC_2_6', 'IC_2_6', 'TN_2_6',
                           'DI_Al_2_6', 'DI_Ca_2_6', 'DI_Fe_2_6', 'DI_K_2_6', 'DI_Mg_2_6', 'DI_P_2_6', 
                           'H3A_Al_2_6', 'H3A_Ca_2_6', 'H3A_Fe_2_6', 'H3A_K_2_6', 'H3A_Mg_2_6', 'H3A_P_2_6', 
                           'M3_Al_2_6', 'M3_Ca_2_6', 'M3_Fe_2_6', 'M3_K_2_6', 'M3_Mg_2_6', 'M3_P_2_6',
                           'Ols_Al_2_6', 'Ols_Ca_2_6', 'Ols_Fe_2_6', 'Ols_K_2_6', 'Ols_Mg_2_6', 'Ols_P_2_6',
                           'Rainfall', 'PhosphorusTreatment',
        'Zone_1', 'Zone_2', 'Zone_3', 'Zone_4']]

#'TC_0_2', 'TC_2_6', 'elevation', 'slope','aspect', 

In [4]:
# Define your features and the column to predict
feature_cols = df.columns[df.columns.get_loc('TOC_0_2'):]
output_col = 'Yield'

# Define the grid of parameters to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# For each year and soil test, train a random forest and save the model and results
X = df[feature_cols]
y = df[output_col]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit model
rf = RandomForestRegressor()
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get best model
best_model = grid_search.best_estimator_

# Compute metrics on test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Save results
results = {
    'model_name': f'Full_model',
    'num_input_params': len(feature_cols),
    'r2': r2,
    'mse': mse,
    'features_importances': {feature: importance for feature, importance in zip(feature_cols, best_model.feature_importances_)},
}
# Save as JSON
with open('Output/Models/rainfall_rf_model_results.json', 'w') as f:
  json.dump(results, f)

['best_model_Full_model.joblib']

In [6]:
# Write code to read in best param files and perform random forest
# Get best model
# Load model
best_model = load('Output/Models/best_model_Full_model.joblib')

# Get best parameters
best_params = best_model.get_params()

feature_cols = df.columns[df.columns.get_loc('TOC_0_2'):]
output_col = 'Yield'

# For each year and soil test, train a random forest and save the model and results
X = df[feature_cols]
y = df[output_col]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute metrics on test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Save results
results = {
    'model_name': f'Full_model',
    'num_input_params': len(feature_cols),
    'r2': r2,
    'mse': mse,
    'features_importances': {feature: importance for feature, importance in zip(feature_cols, best_model.feature_importances_)},
}

In [11]:
# Open JSON file
with open('Output/Models/rainfall_rf_model_results.json', 'r') as f:
  results = json.load(f)

In [12]:
results

{'model_name': 'Full_model',
 'num_input_params': 60,
 'r2': 0.8865036735240885,
 'mse': 195.82341644875433,
 'features_importances': {'TOC_0_2': 0.005827177980144972,
  'IC_0_2': 0.008363083576724684,
  'TN_0_2': 0.0059911211232614005,
  'DI_Al_0_2': 0.004419378284654296,
  'DI_Ca_0_2': 0.0053555546964036155,
  'DI_Fe_0_2': 0.004322760720884275,
  'DI_K_0_2': 0.004609431328310892,
  'DI_Mg_0_2': 0.004091534667906089,
  'DI_P_0_2': 0.003700073059937004,
  'H3A_Al_0_2': 0.00613389735014408,
  'H3A_Ca_0_2': 0.004518177287128801,
  'H3A_Fe_0_2': 0.018812413903013678,
  'H3A_K_0_2': 0.004751324539084814,
  'H3A_Mg_0_2': 0.006543992087198581,
  'H3A_P_0_2': 0.005613744440193842,
  'M3_Al_0_2': 0.01446582283530297,
  'M3_Ca_0_2': 0.02171987269424942,
  'M3_Fe_0_2': 0.015974837030354957,
  'M3_K_0_2': 0.005875363832229931,
  'M3_Mg_0_2': 0.005316611951095211,
  'M3_P_0_2': 0.004574998091265745,
  'Ols_Al_0_2': 0.004078676256882419,
  'Ols_Ca_0_2': 0.008047999339344275,
  'Ols_Fe_0_2': 0.00642

In [13]:
results['r2']*results['features_importances']['Rainfall']

0.49893977378954424

In [14]:
results['r2']*results['features_importances']['PhosphorusTreatment']

0.0075076325882090095

In [28]:
# # Define a dictionary mapping the 'Year' to the corresponding mean temp (°C) value
# temp_data = {2018: 34.0, 2019: 28.2, 2020: 30.5, 2022: 34.4}

# # Use the 'Year' column to map the corresponding 'Rainfall' value and create a new column
# df['Temp'] = df['Year'].map(temp_data)